In [42]:
import xmltodict
from bs4 import BeautifulSoup
import requests
import string
import nltk
import gensim
response = requests.get('https://www.mwnation.com/feed/?paged=501')
doc = xmltodict.parse(response.text)
d = doc['rss']['channel']['item']
docs = list()
for i in d:
    #print(i)
    content = i['content:encoded']
    soup = BeautifulSoup(content, 'html.parser')
    docs.append(soup.get_text())
#print(docs)

from nltk.tokenize import word_tokenize
#return tokens
token_docs = [word_tokenize(i.encode('ascii',errors='ignore').decode()) for i in docs]

word_docs = list()
#remove punctuation
for token_doc in token_docs:
    word_docs.append([word for word in token_doc if word.isalpha()])
from nltk.corpus import stopwords
extra_stops = ['ine','wanga','inemwini','ife','zathu','tokha','inu','anu','wanu','nokha','iye','ake','iyemwini','izo',
'iwo','awo','zawo','iwowo','chani','zomwe','who','amene','izi','kuti','amenewo','ndili','ndi','anali','khalani','akhala',
'kukhala','ali','chitani','amatero','anatero','akuchita','a','koma','ngati','kapena','chifukwa','monga','mpaka','pomwe',
'wa','at','za','motsutsa','pakati','mu','kudzera','nthawi','kale','pambuyo','pamwambapa','pansipa','kuchokera',"m'mwamba",
'pansi','kunja','pa','zochotsa','kwatha','kachiwiri','kupitirira','ndiye','kamodzi','Pano','Apo','liti','pati','bwanji',
'zonse','chilichonse','onse','aliyense','ochepa','Zambiri','ambiri','ena','zotere','ayi','ngakhale','kokha','changa',
'momwemonso','choncho','kuposa','nawonso','kwambiri','angathe','chifuniro','basi','don','ayenera','tsopano',]
stop_words = set(stopwords.words('english'))
stop_words.update(extra_stops)
nostw_docs = list()
#remove stopwords
for word_doc in word_docs:
    nostw_docs.append([w for w in word_doc if not w in stop_words])
no1ltw_docs = list()
#remove 1 letter words and lowercase
for nostw_doc in nostw_docs:
    no1ltw_docs.append([w.lower() for w in nostw_doc if not len(w) <= 2])

bigram = gensim.models.Phrases(no1ltw_docs, min_count=1, threshold=1)
bigram_mod = gensim.models.phrases.Phraser(bigram)

bigram_docs = list()
for no1ltw_doc in no1ltw_docs:
    bigram_docs.append(bigram_mod[no1ltw_doc])

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemma_docs = list()
for bigram_doc in bigram_docs:
    lemma_docs.append([lemmatizer.lemmatize(word) for word in bigram_doc])

from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
stemmed_docs = list()
for lemma_doc in lemma_docs:
    stemmed_docs.append([stemmer.stem(word) for word in lemma_doc])
    

import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

id2word = corpora.Dictionary(stemmed_docs)
corpus = [id2word.doc2bow(text) for text in stemmed_docs]
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=10,chunksize = 50,update_every=1,
                                           passes=10000,alpha='auto',per_word_topics=True)
from pprint import pprint
pprint(lda_model.print_topics())
# Compute Perplexity
print('\nModel Perplexity: ', lda_model.log_perplexity(corpus))  #lower is better
coherence_model_lda = CoherenceModel(model=lda_model, texts=stemmed_docs, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nModel Coherence: ', coherence_lda)#higher is better

#import pyLDAvis
#import pyLDAvis.gensim
#import matplotlib.pyplot as plotter
#%matplotlib inline
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#vis

[(0,
  '0.016*"aphunzitsi" + 0.013*"bingu" + 0.011*"boma" + 0.011*"statu" + '
  '0.009*"presid" + 0.007*"unveil" + 0.006*"mutharika" + 0.006*"kampeni" + '
  '0.006*"the" + 0.006*"tum"'),
 (1,
  '0.007*"anthu" + 0.006*"said" + 0.005*"dream" + 0.005*"inn" + 0.005*"say" + '
  '0.004*"adatero" + 0.004*"gogo" + 0.004*"adati" + 0.004*"imposs" + '
  '0.004*"makolo"'),
 (2,
  '0.013*"madzi" + 0.012*"mbewu" + 0.010*"ulimiwu" + 0.010*"kodi" + '
  '0.008*"mvula" + 0.008*"nanga" + 0.007*"ndipo" + 0.005*"chimanga" + '
  '0.005*"moter" + 0.005*"komanso"'),
 (3,
  '0.012*"qualifi" + 0.012*"flame" + 0.007*"executive_committe" + 0.007*"cup" '
  '+ 0.007*"respons" + 0.007*"said" + 0.005*"morocco" + 0.005*"submit" + '
  '0.005*"made" + 0.005*"waiting_govern"'),
 (4,
  '0.011*"candid" + 0.010*"said" + 0.008*"mbewu" + 0.007*"team" + '
  '0.006*"malawi" + 0.006*"govern" + 0.005*"tackl" + 0.005*"would" + '
  '0.005*"debat" + 0.004*"issu"'),
 (5,
  '0.017*"grief" + 0.013*"loss" + 0.010*"person" + 0.008*"life"